In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from gcspy import GraphOfConvexSets

In [ ]:
mesh = np.array([
    [[1, 0], [0, 2], [0, 1]],
    [[1, 0], [0, 2], [1, 3]],
    [[1, 0], [2, 0], [1, 3]],
    [[3, 1], [2, 0], [1, 3]],
    [[3, 1], [3, 3], [1, 3]],
    [[3, 1], [3, 3], [5, 3]],
    [[3, 1], [5, 3], [5, 1]],
    [[7, 1], [5, 3], [5, 1]],
    [[7, 3], [5, 3], [7, 1]],
    [[7, 3], [9, 3], [7, 1]],
    [[9, 3], [9, 1], [7, 1]],
    [[13, 1], [9, 3], [9, 1]],
    [[13, 1], [9, 3], [14, 3]],
    [[10, 5], [9, 3], [14, 3]],
    [[10, 5], [14, 5], [14, 3]],
    [[10, 5], [14, 5], [11, 6]],
    [[13, 6], [14, 5], [11, 6]],
])

x_min = np.full(2, np.inf)
x_max = - x_min
for triangle in mesh:
    x_min = np.min([x_min, np.min(triangle, axis=0)], axis=0)
    x_max = np.max([x_max, np.max(triangle, axis=0)], axis=0)
r_max = np.linalg.norm(x_max - x_min, 2) / 2

def plot_mesh():
    plt.axis('square')
    for triangle in mesh:
        plt.gca().add_patch(plt.Polygon(triangle[:3,:], fc='mintcream', ec='k'))
    plt.xlim(x_min[0] - 1, x_max[0] + 1)
    plt.ylim(x_min[1] - 1, x_max[1] + 1)
        
plt.figure()
plot_mesh()

In [ ]:
gcs = GraphOfConvexSets()

# fixed cost of adding a sphere
alpha = 0

n_spheres = 5
spheres = []
for i in range(n_spheres):
    s = gcs.add_vertex(f"s{i}")
    c = s.add_variable(2)
    r = s.add_variable(1)
    s.add_constraint(c >= x_min)
    s.add_constraint(c <= x_max)
    s.add_constraint(r >= 0)
    s.add_constraint(r <= r_max)
    s.add_cost(alpha + r * 0)
    s.add_cost(np.pi * r ** 2)
    spheres.append(s)
    
triangles = []
for i in range(len(mesh)):
    t = gcs.add_vertex(f"t{i}")
    t.add_constraint(t.add_variable(1)[0] == 0)
    triangles.append(t)
    
for s in spheres:
    c = s.variables[0]
    r = s.variables[1]
    for i, t in enumerate(triangles):
        edge = gcs.add_edge(s, t)
        for p in mesh[i]:
            edge.add_constraint(cp.norm(p - c, 2) <= r)

In [ ]:
gcs.graphviz()

In [ ]:
from time import time
tic = time()
prob = gcs.solve_facility_location()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)
print(time() - tic)

In [ ]:
plt.figure()
plt.axis('square')
plt.axis('off')

plot_mesh()

for s in spheres:
    c = s.variables[0].value
    if c is not None:
        r = s.variables[1].value
        plt.gca().add_patch(plt.Circle(c, r, fc='None', ec='b'))
        
plt.xlim(x_min[0] - 1, x_max[0] + 1)
plt.ylim(x_min[1] - 1, x_max[1] + 1)

plt.savefig('cover.pdf', bbox_inches='tight')